<a href="https://colab.research.google.com/github/Davichobacter/frist-repo/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -qq
!apt-get install -y tesseract-ocr tesseract-ocr-spa tesseract-ocr-eng poppler-utils
!pip install -q pytesseract pdf2image pillow opencv-python pypdf

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
The following NEW packages will be installed:
  poppler-utils tesseract-ocr-spa
0 upgraded, 2 newly installed, 0 to remove and 53 not upgraded.
Need to get 1,137 kB of archives.
After this operation, 3,006 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-spa all 1:4.00~git30-7274cfa-1.1 [951 kB]
Fetched 1,137 kB in 1s (1,340 kB/s)
Selecting previou

In [5]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import cv2
import numpy as np
from pypdf import PdfWriter, PdfReader
import os

PDF_IN = "/content/Actividad 1_Lect3_Tortella_Cap5.pdf"
PDF_OUT = "Tortella_Cap5.pdf"
LANG = "spa+eng"
DPI = 300

def preprocess(pil_img: Image.Image) -> Image.Image:
    img = np.array(pil_img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Limpieza ligera
    gray = cv2.medianBlur(gray, 3)

    # Binarización
    _, th = cv2.threshold(
        gray, 0, 255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    return Image.fromarray(th)

pages = convert_from_path(PDF_IN, dpi=DPI)

writer = PdfWriter()
temp_files = []

tess_config = "--oem 1 --psm 3"

for i, page in enumerate(pages, start=1):
    clean = preprocess(page)

    pdf_bytes = pytesseract.image_to_pdf_or_hocr(
        clean,
        lang=LANG,
        config=tess_config,
        extension="pdf"
    )

    tmp_name = f"_tmp_page_{i}.pdf"
    with open(tmp_name, "wb") as f:
        f.write(pdf_bytes)

    reader = PdfReader(tmp_name)
    for p in reader.pages:
        writer.add_page(p)

    temp_files.append(tmp_name)
    print(f"OCR página {i}/{len(pages)} listo")

with open(PDF_OUT, "wb") as f:
    writer.write(f)

# Limpieza
for f in temp_files:
    os.remove(f)

print("✅ PDF buscable generado:", PDF_OUT)


OCR página 1/22 listo
OCR página 2/22 listo
OCR página 3/22 listo
OCR página 4/22 listo
OCR página 5/22 listo
OCR página 6/22 listo
OCR página 7/22 listo
OCR página 8/22 listo
OCR página 9/22 listo
OCR página 10/22 listo
OCR página 11/22 listo
OCR página 12/22 listo
OCR página 13/22 listo
OCR página 14/22 listo
OCR página 15/22 listo
OCR página 16/22 listo
OCR página 17/22 listo
OCR página 18/22 listo
OCR página 19/22 listo
OCR página 20/22 listo
OCR página 21/22 listo
OCR página 22/22 listo
✅ PDF buscable generado: Tortella_Cap5.pdf


In [6]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import cv2
import numpy as np

PDF_IN = "/content/Actividad 1_Lect3_Tortella_Cap5.pdf"
TXT_OUT = "Tortella_Cap6.txt"
LANG = "spa+eng"
DPI = 300

def preprocess(pil_img):
    img = np.array(pil_img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.medianBlur(gray, 3)
    _, th = cv2.threshold(
        gray, 0, 255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )
    return Image.fromarray(th)

pages = convert_from_path(PDF_IN, dpi=DPI)

all_text = []

for i, page in enumerate(pages, start=1):
    clean = preprocess(page)
    text = pytesseract.image_to_string(
        clean,
        lang=LANG,
        config="--oem 1 --psm 3"
    )
    all_text.append(f"\n\n### Página {i}\n{text}")

final_text = "\n".join(all_text)

with open(TXT_OUT, "w", encoding="utf-8") as f:
    f.write(final_text)

print("✅ Texto OCR generado:", TXT_OUT)


✅ Texto OCR generado: Tortella_Cap6.txt


In [7]:
MD_OUT = "Tortella_Cap5.md"

from google.colab import files

with open(MD_OUT, "w", encoding="utf-8") as f:
    f.write("# Contenido OCR del documento\n")
    f.write(final_text)

files.download(MD_OUT)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>